# Train LongFormer

In [5]:
import numpy as np
import pandas as pd 
from tqdm import tqdm 
from sentence_transformers import SentenceTransformer, util, models
import os 
import random
import time
import datetime
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers.optimization import get_linear_schedule_with_warmup, AdamW 
from transformers import LongformerTokenizer, LongformerForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 

import telegram 

tel_tokens = "5167864679:AAHlHd1MnkxkORh1-h3B2DjQI4BuYIBG2qY"
chat_id = 1720119057 
bot = telegram.Bot(token=tel_tokens)

In [40]:
train_val_combined.head()

,queries,positive,negative
0,The invention using the modified hump contour...,A fishing lure comprising an elongated body h...,An integrated circuit comprising a ferroelect...
1,A method comprising a first hydrothermal trea...,A method for hydrothermally processing biomas...,A self destruct frangible projectile for mark...
2,A decimal absolute value adder comprises a fi...,A computer program product for implementing a...,A lubrication system for an axle assembly com...
3,A bi directional valve assembly comprising a ...,A spill proof drinking container comprising a...,A disc brake for a motor vehicle comprising a...
4,An isolated adipose tissue derived stromal ce...,An isolated adipose tissue derived stromal ce...,A switching mode power supply comprising a po...


In [41]:
train_val_combined.shape

(350399, 3)

In [46]:
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
input_ids, attention_masks, labels = [], [], [] 
q = queries[0]
for i in tqdm(range(candidates.shape[0])):
    query_tokens = tokenizer.encode(q) 
    query_tokens = query_tokens[1:-1] 

    candidate_tokens = tokenizer.encode(candidates[0])
    
    negative_tokens = tokenizer.encode(negative_doc) 
    negative_tokens = negative_tokens[1:-1] 

    # process positive data 
    input_tokens = [0] + query_tokens[:2046] + [2] + [2] + positive_tokens[:2046] + [2] 
    pad_length = 4096 - len(input_tokens) 
    input_tokens = input_tokens + [1 for _ in range(pad_length)] 

    attention_mask = [] 
    for t in input_tokens:
        if t != 1:
            attention_mask.append(1) 
        elif t == 1:
            attention_mask.append(0) 
            
    input_ids.append(input_tokens) 
    attention_masks.append(attention_mask)
    labels.append(1) 
    
    # process negative data 
    input_tokens_neg = [0] + query_tokens[:2046] + [2] + [2] + negative_tokens[:2046] + [2] 
    pad_length = 4096 - len(input_tokens_neg) 
    input_tokens_neg = input_tokens_neg + [1 for _ in range(pad_length)] 

    attention_mask_neg = [] 
    for t in input_tokens_neg:
        if t != 1:
            attention_mask_neg.append(1) 
        elif t == 1:
            attention_mask_neg.append(0) 
            
    input_ids.append(input_tokens_neg) 
    attention_masks.append(attention_mask_neg)
    labels.append(0) 
    

In [137]:
train_input_ids, val_input_ids, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.2, stratify=labels, random_state=42)

train_attention_masks, val_attention_masks, _, _ = train_test_split(attention_masks, labels, test_size=0.2, stratify=labels, random_state=42)


In [138]:
train_input_ids = torch.tensor(train_input_ids, dtype=int) 
val_input_ids = torch.tensor(val_input_ids, dtype=int) 
train_labels = torch.tensor(train_labels, dtype=int) 
val_labels = torch.tensor(val_labels, dtype=int) 
train_attention_masks = torch.tensor(train_attention_masks, dtype=int) 
val_attention_masks = torch.tensor(val_attention_masks, dtype=int)  

train_input_ids.shape, val_input_ids.shape, train_labels.shape, val_labels.shape, train_attention_masks.shape, val_attention_masks.shape

(torch.Size([560638, 4096]),
 torch.Size([140160, 4096]),
 torch.Size([560638]),
 torch.Size([140160]),
 torch.Size([560638, 4096]),
 torch.Size([140160, 4096]))

In [143]:
# torch.save(train_input_ids,'train_input_ids.pt') 

In [144]:
#torch.save(val_input_ids, 'val_input_ids.pt') 
#torch.save(train_labels, 'train_labels.pt') 
#torch.save(val_labels, 'val_labels.pt') 
#torch.save(train_attention_masks, 'train_attention_masks.pt') 
#torch.save(val_attention_masks, 'val_attention_masks.pt')

In [2]:
train_input_ids = torch.load('train_input_ids.pt') 
val_input_ids = torch.load('val_input_ids.pt') 
train_labels = torch.load('train_labels.pt') 
val_labels = torch.load('val_labels.pt') 
train_attention_masks = torch.load('train_attention_masks.pt') 
val_attention_masks = torch.load('val_attention_masks.pt')

In [3]:
train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels) 
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=1) 

s = 40000

validation_data = TensorDataset(val_input_ids[:s], val_attention_masks[:s], val_labels[:s]) 
validation_sampler = RandomSampler(validation_data) 
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=1)


# Train LongFormer

In [4]:
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096") 
model.cuda() 

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 

LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0): LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (val

In [5]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8) 
epochs = 1
total_steps = len(train_dataloader) * epochs 

'''
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
''' 

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


'\nscheduler = get_linear_schedule_with_warmup(optimizer, \n                                            num_warmup_steps = 0,\n                                            num_training_steps = total_steps)\n'

In [6]:
def flat_accuracy(preds, labels): 
    pred_flat = np.argmax(preds, axis=1).flatten() 
    labels_flat = labels.flatten() 
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def validate(validation_dataloader):
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in tqdm(validation_dataloader, desc="Iteration", smoothing=0.05, position=0, leave=True):
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels = b_labels)
    
        
        loss = outputs[0] 
        logits = outputs[1] 
        
        eval_loss += loss.item() 
        
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.cpu().numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    
    avg_val_loss = eval_loss / len(validation_dataloader)   
    avg_val_accuracy = eval_accuracy / len(validation_dataloader) 
    
    val_losses.append(avg_val_loss)
    val_accuracies.append(avg_val_accuracy)
    
    print("  Average validation loss: {0:.2f}".format(avg_val_loss))
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
    
    text = "  Average validation loss: {0:.2f}".format(avg_val_loss) 
    bot.sendMessage(chat_id=chat_id, text=text) 
    text = "  Accuracy: {0:.2f}".format(avg_val_accuracy)
    bot.sendMessage(chat_id=chat_id, text=text)
    text = "  Validation took: {:}".format(format_time(time.time() - t0)) 
    bot.sendMessage(chat_id=chat_id, text=text) 
    
    
    if np.min(val_losses) == val_losses[-1]: 
        print("saving best checkpoint...")
        text = "saving best checkpoint..." 
        bot.sendMessage(chat_id=chat_id, text=text)
        torch.save(model.state_dict(), "patent_LongFormer.pt".format(avg_val_accuracy, avg_val_loss)) 


In [ ]:
device = torch.device("cuda")

accum_iter = 32 

print("training on {}...".format(device))

bot.sendMessage(chat_id=chat_id, text="training on {}...".format(device))

train_losses, val_losses = [], [] 
train_accuracies, val_accuracies = [], [] 

model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0
    total_accuracy = 0 

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration", smoothing=0.05, position=0, leave=True)):
        # 경과 정보 표시
        if step % 10000 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print('  current average loss = {}'.format(total_loss / step))
            print('  current average accuracy = {}'.format(total_accuracy / step))
            
            text = '  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed)
            bot.sendMessage(chat_id=chat_id, text=text)
                                                                         
            text = '  current average loss = {}'.format(total_loss / step)
            bot.sendMessage(chat_id=chat_id, text=text) 
                                                                         
            text = '  current average accuracy = {}%'.format(total_accuracy / step)
            bot.sendMessage(chat_id=chat_id, text=text) 
                                                                         
            validate(validation_dataloader)

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]
        logits = outputs[1] 
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.detach().cpu().numpy() 

        total_loss += loss.item()
        total_accuracy += flat_accuracy(logits, label_ids) 

        loss.backward()
        
        if ((step + 1) % accum_iter == 0) or (step + 1 == len(train_dataloader)):
            # weights update 
            optimizer.step()
            model.zero_grad() 
    
    
    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)      
    avg_train_accuracy = total_accuracy / len(train_dataloader) 
    
    train_losses.append(avg_train_loss) 
    train_accuracies.append(avg_train_accuracy) 
    
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Average training accuracy: {0:.2f}".format(avg_train_accuracy))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
print("")
print("Training complete!")


training on cuda...

======== Epoch 1 / 1 ========
Training...


Iteration:   1%|▏         | 8317/560638 [1:28:14<96:52:36,  1.58it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Iteration:  11%|█▏        | 4511/40000 [13:25<1:45:47,  5.59it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Iteration:  26%|██▌       | 10346/40000 [30:43<1:29:49,  5.50it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this lim

  Batch 20,000  of  560,638.    Elapsed: 5:28:52.
  current average loss = 0.5647738727882504
  current average accuracy = 0.69015

Running Validation...


Iteration:   7%|▋         | 2977/40000 [08:51<1:47:37,  5.73it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Iteration:  23%|██▎       | 9280/40000 [27:33<1:31:58,  5.57it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Iteration:  34%|███▍      | 13724/40000 [40:43<1:17:19,  5.66it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, s

  Average validation loss: 0.43
  Accuracy: 0.82
  Validation took: 1:59:01


...

Iteration:  25%|██▌       | 140562/560638 [51:45:00<72:14:42,  1.62it/s]    IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Iteration:  27%|██▋       | 150000/560638 [53:22:17<70:50:16,  1.61it/s]

  Batch 150,000  of  560,638.    Elapsed: 2 days, 5:22:18.
  current average loss = 0.309479562874611
  current average accuracy = 0.8718866666666667

Running Validation...


Iteration:  15%|█▌        | 6150/40000 [17:49<1:36:17,  5.86it/s]]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Iteration:  30%|███       | 170000/560638 [60:42:44<67:05:33,  1.62it/s]

  Batch 170,000  of  560,638.    Elapsed: 2 days, 12:42:44.
  current average loss = 0.2999718347497976
  current average accuracy = 0.8777470588235294

Running Validation...


Iteration:  23%|██▎       | 9045/40000 [26:34<1:29:38,  5.76it/s]

# Inference

In [6]:
test_model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096") 
checkpoint = torch.load("patent_LongFormer.pt") 
test_model.load_state_dict(checkpoint) 
test_model.cuda() 
test_model.eval()

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 

LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0): LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (val

In [7]:
test_df = pd.read_csv("test_df.tsv", sep="\t") 
test_df.head()

,queries,positive,negative
0,A system for charging or maintaining a batter...,A method for battery charger and diagnosis wi...,A ductile failure anchor for a concrete eleme...
1,A method for calculating a path delay in stat...,A non transitory computer readable medium car...,A distance evaluation method for evaluating d...
2,A preventive or therapeutic agent for diabete...,A method for measuring glutamyl transpeptidas...,A communication system comprising a plurality...
3,A semiconductor device comprising a first sem...,An integrated circuit comprising a first pitc...,canceled A lighting system comprising an opti...
4,A moveable barrier operator system comprising...,A movable barrier operator having automatic f...,A remote control system which comprises a rem...


In [9]:
# load dictionary with labels 
import pickle 

with open('query_positive_dict.pkl', 'rb') as f:
    d = pickle.load(f)

len(d)

32764

In [14]:
candidates = np.concatenate([test_df['positive'].values, 
                             test_df['negative'].values]) 

candidates.shape

(77886,)

In [16]:
queries = test_df['queries'].values 

In [21]:
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

In [53]:
input_ids, attention_masks = [], [] 

device = torch.device('cuda')

m = nn.Softmax(dim=1)

final_ranks = [] 

for q in queries[:10]: 
    ranks = [] 
    for i in tqdm(range(10000)):
        query_tokens = tokenizer.encode(q) 
        query_tokens = query_tokens[1:-1] 

        candidate_tokens = tokenizer.encode(candidates[i]) 
        candidate_tokens = candidate_tokens[1:-1] 

        # process positive data 
        input_tokens = [0] + query_tokens[:2046] + [2] + [2] + candidate_tokens[:2046] + [2] 
        pad_length = 4096 - len(input_tokens) 
        input_tokens = input_tokens + [1 for _ in range(pad_length)] 

        attention_mask = [] 
        for t in input_tokens:
            if t != 1:
                attention_mask.append(1) 
            elif t == 1:
                attention_mask.append(0) 

        input_tokens = torch.tensor(input_tokens, dtype=int) 
        input_tokens = torch.reshape(input_tokens, (-1, len(input_tokens))) 
        input_tokens = input_tokens.to(device)

        attention_mask = torch.tensor(attention_mask, dtype=int) 
        attention_mask = torch.reshape(attention_mask, (-1, len(attention_mask))) 
        attention_mask = attention_mask.to(device) 

        with torch.no_grad(): 
            output = test_model(input_tokens, 
                           token_type_ids=None, 
                           attention_mask=attention_mask) 

        logits = output[0] 
        probs = m(logits) 
        probs = probs.detach().cpu().numpy().flatten() 
        ranks.append((i+1, probs[1])) 
    ranks = sorted(ranks, key=lambda x: x[1], reverse=True) 
    # print(ranks) 
    for idx, v in enumerate(ranks):
        if v[0] in d[q]:
            print("rank : {}, predicted score = {}".format(idx+1, v[1]))
            final_ranks.append(idx+1) 
            break                          

100%|██████████| 10000/10000 [35:50<00:00,  4.65it/s] 


rank : 11, predicted score = 0.9653783440589905


100%|██████████| 10000/10000 [36:26<00:00,  4.57it/s] 


rank : 75, predicted score = 0.9368259310722351


  2%|▏         | 168/10000 [00:36<34:41,  4.72it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 10000/10000 [36:21<00:00,  4.58it/s] 


...

rank : 31, predicted score = 0.9012483954429626


100%|██████████| 10000/10000 [34:13<00:00,  4.87it/s] 


rank : 15, predicted score = 0.9477647542953491


100%|██████████| 10000/10000 [32:39<00:00,  5.10it/s] 

rank : 9, predicted score = 0.9255420565605164


In [54]:
final_ranks

[11, 75, 76, 24, 1, 34, 12, 31, 15, 9]

In [55]:
mrr = 0 

for r in final_ranks:
    mrr += 1/r 
    
mrr /= len(final_ranks) 

print("MRR = {}".format(mrr))

MRR = 0.14818479259790554
